In [ ]:
from pyciss.io import RingCube, PathManager, db_mapped_cubes

In [ ]:
pm = PathManager("N1467345444") 

# meta development

In [ ]:
import pkg_resources as pr

In [ ]:
with pr.resource_stream('pyciss', 'data/ring_resonances.csv') as f:
    resonances = pd.read_csv(f)
resonances.head()

In [ ]:
resonances.sort_values(by='Resonance (km)',ascending=False).head()

In [ ]:
from pyciss.meta import meta_df

In [ ]:
meta_df.head()

In [ ]:
meta_df.loc[pm._id]

In [ ]:
cube = RingCube(pm.cubepath)

In [ ]:
cube.meta.pixres

In [ ]:
%matplotlib nbagg

In [ ]:
cube.imshow()

In [ ]:
cube.meta

In [ ]:
dics = []
for p in db_mapped_cubes():
    d = {}
    d['path'] = p
    cube = RingCube(p)
    d['meta_res'] = cube.meta_pixres
    d['label_res'] = cube.resolution_val
    d['time'] = cube.imagetime
    dics.append(d)

In [ ]:
df = pd.DataFrame(dics)

In [ ]:
df.head()

In [ ]:
df['id'] = df.path.map(lambda x: x.name.split('_')[0])

In [ ]:
df.set_index('time',inplace=True)

In [ ]:
df.set_index('id', inplace=True)

In [ ]:
df = df.sort_index()

In [ ]:
df.tail()

In [ ]:
df['ratio'] = df.meta_res / df.label_res

In [ ]:
df[df.label_res<5000].plot(style='*')

In [ ]:
cube = RingCube(df[df.label_res>5000].path.values[0])

In [ ]:
cube.imshow()

In [ ]:
cube.imagetime

In [ ]:
cube.get_opus_meta_data()

In [ ]:
cube.opusmeta.ring_geom

# Resonances plotting

In [ ]:
from pyciss.meta import prime_resonances as resonances
from pyciss import io

In [ ]:
resonances.head()

In [ ]:
all_mapped_cubes = io.db_mapped_cubes()

In [ ]:
%matplotlib nbagg

In [ ]:
cube = io.RingCube(next(all_mapped_cubes))

In [ ]:
cube.imshow()

In [ ]:
cube = io.RingCube(next(all_mapped_cubes))

fig, ax = plt.subplots(figsize=(12,9))
cube.imshow(ax=ax)

ax2 = ax.twinx()
ax2.set_ybound(cube.minrad, cube.maxrad)
ax2.ticklabel_format(useOffset=False)
ax2.set_yticks(newticks.radius/1000)
ax2.set_yticklabels(newticks.name);

In [ ]:
ax.get_ybound()

In [ ]:
ax.get_ylim()

In [ ]:
ax.get_ylim??

In [ ]:
ax.get_ybound??

In [ ]:
ax2 = ax.twinx

In [ ]:
ax.set_yticks(resonanc)

In [ ]:
f = lambda x,y: (resonances['radius']>x) & (resonances['radius']<y)

In [ ]:
resonances[f(134220, 134326)]

In [ ]:
resonances.describe()